In [2]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import pytorch_lightning as pl
import torchmetrics
import matplotlib.pyplot as plt
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [3]:
train_df = pd.read_csv('./train_data.csv')
val_df = pd.read_csv('./validation_data.csv')
test_df = pd.read_csv('./test_data.csv')
feature_cols = ['Packet speed', 'Data speed', 'Packet Rate', 'Data Rate', 'Packet Size to Speed Ratio', 'Data to Packet Ratio',
                'Duration', 'Hour sin', 'Hour cos', 'Day of Week','Unique ports', 'Std packet speed', 'Max packet speed', 
                'Std data speed', 'Max data speed', 'Std packet length', 'Max packet length',
                'Std source IP count', 'Max source IP count', 'Mean time diff', 'Std time diff','pca_1','pca_2']

features_selected = ['Data to Packet Ratio', 'Packet Size to Speed Ratio', 'Packet speed', 'Data speed', 'Max packet length', 'Hour cos', 'Day of Week', 'Std packet length', 'Unique ports']
X_train = train_df[features_selected]
y_train = train_df['Type']
X_val = val_df[features_selected]
y_val = val_df['Type']
X_test = test_df[features_selected]
y_test = test_df['Type']
le = LabelEncoder()
y_test = le.fit_transform(y_test)
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
scaler = StandardScaler()

### 8. futás (legmagasabb test accuracy)

In [4]:
batch_size = 64
dropout_prob = 0.2
hidden_dim = 32
lr = 0.00013911830625926386
num_epochs = 12

In [5]:
def create_dataloader(X, y, batch_size):
  scaler.fit(X)
  X = scaler.transform(X).astype('float32')
  X = torch.from_numpy(X)
  y = y.astype('long')
  y = torch.from_numpy(y)
  dataset = TensorDataset(X, y)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers = 15)
  return dataloader

train_loader = create_dataloader(X_train, y_train, batch_size)
val_loader = create_dataloader(X_val, y_val, batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size)

In [6]:
class FeedForwardNet(pl.LightningModule):
    def __init__(self, num_features, hidden_dim, lr, num_classes=3, dropout_prob=0.3):
        super(FeedForwardNet, self).__init__()
        self.save_hyperparameters()
        self.lr = lr
        self.num_classes = num_classes

        self.layers = nn.Sequential(
            nn.Linear(num_features, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_prob),

            nn.Linear(hidden_dim, num_classes)  # output layer (logits)
        )

        self.loss = nn.CrossEntropyLoss()
        self.acc = torchmetrics.classification.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x):
        return self.layers(x)

    def training_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"loss": loss, "acc": acc}, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"val_loss": loss, "val_acc": acc}, prog_bar=False)
        return {"val_loss": loss, "val_acc": acc}

    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch)
        self.log_dict({"test_loss": loss, "test_acc": acc})
        return {"test_loss": loss, "test_acc": acc}

    def _shared_eval_step(self, batch):
        x, y = batch
        logits = self(x)
        loss = self.loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = self.acc(preds, y)
        return loss, acc

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [7]:
wandb.login()

wandb: Currently logged in as: baczodomonkos (baczodomonkos-budapesti-m-szaki-s-gazdas-gtudom-nyi-egyetem) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
torch.set_float32_matmul_precision('high')
wandb.finish()

acc,▃▅▆▆▇▆▇▇█▇█▇█▇█▁▂▅▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇
epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▆▆▅▅▄▄▃▃▂▂▂▁▂▁█▇▆▅▅▄▃▃▂▂▂▂▂▃▂▂▁▂▂▁▂▁▁▁▂▂
trainer/global_step,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▆▇▇▇██
acc,0.92188
epoch,0
loss,0.24293
trainer/global_step,1849


In [13]:
model = FeedForwardNet(X_train.shape[1], hidden_dim, lr)
wandb_logger = pl.loggers.WandbLogger(project="ddos_classifier_best", log_model="all")
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")
trainer = pl.Trainer(
    max_epochs=num_epochs,
    precision="16-mixed",
    accelerator="gpu",
    devices=1,
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
wandb.finish()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | layers | Sequential         | 1.6 K  | train
1 | loss   | CrossEntropyLoss   | 0      | train
2 | acc    | MulticlassAccuracy | 0      | train
------------------------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=12` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9113950133323669     │
│         test_loss         │    0.3417748510837555     │
└───────────────────────────┴───────────────────────────┘

acc,▅▆▅▇▅▅▇▆▇▇▆▄▇▇▇▆▅▅▇▅▆█▆▂▇▆▅▁▇▅▇▅▄▅▆▄▄▇▆▅
epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇█████
loss,▇▅█▅▄▅▂▂▂▁▃▃▂▃▃▄▂▂▅▄▃▅▂▃▃▂▃▁▂▄▁▃▂▂▇▄▄▄▃▃
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇████
val_acc,▁▅▅▆▇▇█▇▇▇▇█
val_loss,▂▁▃▅▅▆█▇▆▂▅▇
acc,0.96875
epoch,12
loss,0.11328


In [14]:
all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = model(x)
        preds = torch.argmax(logits, dim=1)

        all_preds.append(preds.cpu())
        all_targets.append(y.cpu())

y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_targets).numpy()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.30      0.44     14733
           1       0.91      1.00      0.95    394482
           2       0.00      0.00      0.00     28424

    accuracy                           0.91    437639
   macro avg       0.60      0.43      0.47    437639
weighted avg       0.85      0.91      0.87    437639



### 13. futás (legalacsonyabb test loss)

In [15]:
batch_size= 256
dropout_prob= 0.4
hidden_dim= 32
lr= 0.00014913786993175047
num_epochs= 5

In [16]:
train_loader = create_dataloader(X_train, y_train, batch_size)
val_loader = create_dataloader(X_val, y_val, batch_size)
test_loader = create_dataloader(X_test, y_test, batch_size)

In [17]:
model = FeedForwardNet(X_train.shape[1], hidden_dim, lr)
wandb_logger = pl.loggers.WandbLogger(project="ddos_classifier_best", log_model="all")
checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_acc", mode="max")
trainer = pl.Trainer(
    max_epochs=num_epochs,
    precision="16-mixed",
    accelerator="gpu",
    devices=1,
    logger=wandb_logger,
    callbacks=[checkpoint_callback],
)
trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)
wandb.finish()

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name   | Type               | Params | Mode 
------------------------------------------------------
0 | layers | Sequential         | 1.6 K  | train
1 | loss   | CrossEntropyLoss   | 0      | train
2 | acc    | MulticlassAccuracy | 0      | train
------------------------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:476: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9130036234855652     │
│         test_loss         │    0.2983245253562927     │
└───────────────────────────┴───────────────────────────┘

acc,▁▅▆▇▅▇▆▆▇▇▆▆▇▆▇▇▇▇▆▇█▇▇▆▇▆▇▇▇▆▇▆▇▇▇▇▇█▇▆
epoch,▁▁▁▁▁▁▁▁▁▁▃▃▃▃▃▃▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆████████
loss,█▇▅▅▄▄▃▄▃▃▂▃▃▃▃▂▂▂▂▁▃▂▂▂▁▁▂▂▁▂▂▂▂▂▂▂▂▂▁▁
test_acc,▁
test_loss,▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
val_acc,▅▂▁▅█
val_loss,█▂▁▁▁
acc,0.92969
epoch,5
loss,0.19718


In [18]:
all_preds = []
all_targets = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = model(x)
        preds = torch.argmax(logits, dim=1)

        all_preds.append(preds.cpu())
        all_targets.append(y.cpu())

y_pred = torch.cat(all_preds).numpy()
y_true = torch.cat(all_targets).numpy()
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.01      0.02     14733
           1       0.92      1.00      0.96    394482
           2       0.61      0.17      0.27     28424

    accuracy                           0.91    437639
   macro avg       0.78      0.39      0.42    437639
weighted avg       0.90      0.91      0.88    437639



In [19]:
# Erre még vissza kell térni, mert ezek nem kifejezetten a legjobbak